In [1]:
import sys
print(sys.executable)

import random
random.seed(0)

/Users/emmanechamkin/miniconda3/envs/advml/bin/python


## help setting my kernel
https://janakiev.com/til/jupyter-virtual-envs/
```(advml) Emmas-MacBook-Pro-2:Project emmanechamkin$ python -m ipykernel install --user --name=advml
Installed kernelspec advml in /Users/emmanechamkin/Library/Jupyter/kernels/advml```

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from config import *

# Work so far
Data here has already been preprocessed (in tweet_extraction) and labeled. I changed tactic and then had MTurk do it for me. 

In [3]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import string
import emoji
from emoji.unicode_codes import UNICODE_EMOJI
import csv
import random

# use keras tokenizer instead
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer 
import json

Using TensorFlow backend.


In [4]:
temp = pd.read_csv(OUTPUT_FILE, encoding="mac-roman")

In [5]:
temp.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.text',
       'Answer.sentiment.label', 'Approve', 'Reject'],
      dtype='object')

In [6]:
temp = temp[temp.AssignmentStatus != "Rejected"]
temp = temp[['HITId', 'Answer.sentiment.label', 'Input.text']]

# take the most common result
final = temp.groupby(['HITId','Input.text']).agg(lambda x:x.value_counts().index[0])

# print to a file
final.to_csv(CLEANED)

In [7]:
## temporary cleaning
def parse(word):
    word = word.strip().lower().translate(TABLE)
    if word in UNICODE_EMOJI:
        word = re.sub("_", " ", UNICODE_EMOJI[word][1:-1]).lower()
    if (word.startswith("rt")
            or word.startswith("@")
            or "http" in word
            or word in STOP_WORDS):
        word = ""
    if word.startswith("#"):
        word = word[1:]
    return word


STOP_WORDS = set(stopwords.words('english'))
TABLE = str.maketrans({key: None for key in string.punctuation})
STEMMER = PorterStemmer()

with open(CLEANED, "r") as csv_file, \
    open(FINAL, "w", newline="") as out_file:
        reader = csv.reader(csv_file, delimiter=",")
        writer = csv.writer(out_file, delimiter=",")
        for row in reader:
            tweet = row[1]
            tweet_keep = ""
            for word in tweet.split():
                # get rid of all non-ascii characters
                word = re.sub(r'[^\x00-\x7f]', r'', word)
                # clean
                word = parse(word)
                # stem
                word_stem = STEMMER.stem(word)
                # save
                if word_stem:
                    tweet_keep += " " + word_stem
            if tweet_keep:
                try:
                    writer.writerow([row[0], tweet_keep, row[2]])
                except Exception as e: #add error here
                    print("Error", e)

In [8]:
# read in the data 
tweet_data = pd.read_csv(FINAL, index_col=0)
tweet_data.head()

,inputtext,Answer.sentiment.label
HITId,,
301KG0KX9CMDD7S33IQR6Z489RA2HI,artsprison williamay convers wthe author tort...,About the police - Positive
301KG0KX9CMDD7S33IQR6Z489RAH2X,imalibrarian chicagosmayor dad lifelong chica...,About the police - Neutral
301KG0KX9CMDD7S33IQR6Z489RB2HJ,chicago polic swat offic respond report arm i...,About the police - Neutral
302OLP89DZ8B12CRU5551QQQ8NXACO,almost forgot foia turn one year old today hb...,Not about the police
302OLP89DZ8B12CRU5551QQQ8NXCAQ,three peopl charg miss pregnant woman found d...,Not about the police


In [9]:
tweet_data.iloc[:, 1].value_counts()

About the police - Neutral     541
Not about the police           442
About the police - Negative    290
About the police - Positive    264
Name: Answer.sentiment.label, dtype: int64

In [10]:
### train / validate / test split 
tweet_id = set(tweet_data.index)
total = len(tweet_id)
train_frac = 0.5
validate_frac = 0.5
# I ended up using validate as test for normal, and built-in 
# validation for NNs, so I increased this so my models would work :)

train_size = int(total*train_frac)
validate_size = int(total*validate_frac)

train_id = random.sample(tweet_id, train_size)
tweet_id = tweet_id.difference(train_id)

validate_id = random.sample(tweet_id, validate_size)
tweet_id = tweet_id.difference(validate_id)

test_id = tweet_id

train_df = tweet_data[tweet_data.index.isin(train_id)]
validate_df = tweet_data[tweet_data.index.isin(validate_id)]
test_df = tweet_data[tweet_data.index.isin(test_id)]

In [11]:
# for data hygeine -- to check that this 
# is the same over time
tweet_sentiment = train_df.iloc[:, 1]
tweet_sentiment.value_counts()

About the police - Neutral     273
Not about the police           223
About the police - Positive    138
About the police - Negative    134
Name: Answer.sentiment.label, dtype: int64

Please note that the TRAIN set is pretty big -- I know it's bigger than we would traditionally want. Because I ended up using a lot of CV and validation inside of the NN's, I had to split data less for the NNs and wanted to ensure that my models had maximum data. 

Also, my models were unfortunately so bad that this didn't end up being a huge problem...

In [12]:
train_df.to_csv(TRAIN, index=False)
validate_df.to_csv(VALIDATE, index=False)
test_df.to_csv(TEST, index=False)